# Effect of Travel on Baseball Performance
***
**Team Members:** Brandon Zink, Cameron Connor, Abiel Fattore

In this project, we will look at the effect of travel in terms of distance in miles and number of time zones changed on the traveling teams performance.

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline
from math import sin, cos, sqrt, atan2, radians
import math
from scipy import stats

In [3]:
#Read in the data, store in appropriate data frames

dfStadiumsInfo = pd.read_csv('data/Stadium_Info.csv')

#Import and clean the game logs data
dfGameInfo = pd.read_csv('data/Game_Logs.csv', low_memory=False, header=None)
#Name all of the columns
dfGameInfo.columns = (['date','gameNumber','day','awayTeam','awayTeamLg','awayTeamGmNmbr','homeTeam','homeTeamLg','homeTeamGmNmbr','awayScore','homeScore','lengthInOuts','timeOfDay','completion','forfeit','protest','parkID','attendance','lengthOfGameInMinutes','awayLineScore','homeLineScore','awayAB','awayH','away2B','away3B','awayHR','awayRBI','awaySH','awaySF','awayHBP','awayBB','awayIBB','awaySO','awaySB','awayCS','awayGIDP','awayCatchInf','awayLOB','awayPitchersUsed','awayIndivER','awayER','awayWildPitch','awayBalk','awayPO','awayAssists','awayE','awayPassedBalls','awayDoubPlay','awayTripPlay','homeAB','homeH','home2B','home3B','homeHR','homeRBI','homeSH','homeSF','homeHBP','homeBB','homeIBB','homeSO','homeSB','homeCS','homeGIDP','homeCatchInf','homeLOB','homePitchersUsed','homeIndivER','homeER','homeWildPitch','homeBalk','homePO','homeAssists','homeE','homePassedBalls','homeDoubPlay','homeTripPlay','homePlateUmpID','homePlateUmpName','1BUmpID','1BUmpName','2BUmpID','2BUmpName','3BUmpID','3BUmpName','LFUmpID','LFUmpName','RFUmpID','RFUmpName','awayManagerID','awayManagerName','homeManagerID','homeManagerName','winPitcherID','winPitcherName','losePitcherID','losePitcherName','savePitcherID','savePitcherName','GWRBIHitterID','GWRBIHitterName','awaySPID','awaySPName','homeSPID','homeSPName','away1ID','away1Name','away1POS','away2ID','away2Name','away2POS','away3ID','away3Name','away3POS','away4ID','away4Name','away4POS','away5ID','away5Name','away5POS','away6ID','away6Name','away6POS','away7ID','away7Name','away7POS','away8ID','away8Name','away8POS','away9ID','away9Name','away9POS','home1ID','home1Name','home1POS','home2ID','home2Name','home2POS','home3ID','home3Name','home3POS','home4ID','home4Name','home4POS','home5ID','home5Name','home5POS','home6ID','home6Name','home6POS','home7ID','home7Name','home7POS','home8ID','home8Name','home8POS','home9ID','home9Name','home9POS','addInfo','infoAquisition'])
#Remove the last two digits of parkID since they are useless
dfGameInfo['parkID'] = dfGameInfo['parkID'].astype(str).str[:-2].astype(str)
#Get rid of games in Tokyo, Montreal, Puerto Rico, Disney World, Sydney, Fort Bragg
dfGameInfo = dfGameInfo[dfGameInfo['parkID'] != 'TOK']
dfGameInfo = dfGameInfo[dfGameInfo['parkID'] != 'MON']
dfGameInfo = dfGameInfo[dfGameInfo['parkID'] != 'SJU']
dfGameInfo = dfGameInfo[dfGameInfo['parkID'] != 'LBV']
dfGameInfo = dfGameInfo[dfGameInfo['parkID'] != 'SYD']
dfGameInfo = dfGameInfo[dfGameInfo['parkID'] != 'FTB']

dfGameInfo['timeOfDay'].head()

2    D
3    D
4    N
6    D
7    D
Name: timeOfDay, dtype: object

In [4]:
#Team ID Lookup
#This is used across multiple functions to give each team a specific number that can be used in lookups (returns 0 through 29, else -1)

def team_ID_lookup(teamID):
    teamMatrix = ['ANA','ARI','ATL','BAL','BOS','CHN','CHA','CIN','CLE','COL','DET','FLO','HOU','KCA','LAN','MIL','MIN','WAS','NYN','NYA','OAK','PHI','PIT','SLN','SDN','SFN','SEA','TBA','TEX','TOR']
    for i in range (0,30):
        if(teamMatrix[i] == teamID):
            return i
    return -1

In [5]:
#Find Distance
#This function finds the distance between two sets of lat, long

def find_distance(latitude1, longitude1, latitude2, longitude2):
    # approximate radius of earth in mi
    R = 3959.0

    lat1 = radians(latitude1)
    lon1 = radians(longitude1)
    lat2 = radians(latitude2)
    lon2 = radians(longitude2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


In [6]:
#Get the time zone difference between two stadiums, returns a value between -3 and 3, where -3 means that the team traveled
#3 hours westward, and 3 means that the team traveled 3 hours eastward. Takes inputs as 3 letter code for teams. Returns
#-5 if either team is not found.

def find_time_zone_difference(awayTeam, homeTeam):
    #Convert the dataframe to an array to make it easier to loop through
    distance_array = dfStadiumsInfo.values
    
    homeTeamTZ = 1000
    awayTeamTZ = -1000
    
    #Find the time zones for the two teams
    for i in range(0,30):
        if(distance_array[i][1] == homeTeam):
            homeTeamTZ = distance_array[i][4]
        if(distance_array[i][1] == awayTeam):
            awayTeamTZ = distance_array[i][4]
    
    if(homeTeamTZ - awayTeamTZ > 3 or homeTeamTZ - awayTeamTZ < -3):
        return -5
    else:
        return homeTeamTZ - awayTeamTZ

In [7]:
#Gets the distance stored in the distance matrix, returns 0 if not yet updated

def get_distance_matrix(matrix, team1ID, team2ID):
    if(matrix[team1ID][team2ID] > 0):
        return matrix[team1ID][team2ID]
    elif (matrix[team2ID][team1ID] > 0):
        return matrix[team2ID][team1ID]
    else:
        return 0

In [8]:
#Creates/resets the distance matrix to all 0, returns the matrix

def start_distance_matrix():
    return np.zeros((30, 30), dtype=float)

#initialize the distance matrix
dist_matrix = start_distance_matrix()

In [9]:
#Updates a position in the matrix to the specified distance, returns the input matrix

def update_distance_matrix(matrix, team1ID, team2ID, distance):
    matrix[team1ID][team2ID] = distance
    matrix[team2ID][team1ID] = distance
    return matrix

In [10]:
#This gets the distance between two stadiums from the stored matrix, and if it's not there, it calculates it and stores it
#in there (dist_matrix)

def get_distance(awayTeam, homeTeam):
    global dist_matrix
    
    stadium_info_array = dfStadiumsInfo.values
    distance = 0
    #get the distance between the stadiums from the distance matrix if it has already been updated
    distance = get_distance_matrix(dist_matrix, team_ID_lookup(awayTeam), team_ID_lookup(homeTeam))
    
    #if it has not been updated yet, update it
    if(distance == 0):
        #loop through the stadium matrix to find the correct row, get the lat and long from that row, calculate the distance,
        #and then update the matrix to include that distance
        awayLat = awayLong = homeLat = homeLong = -1.0
        for i in range(0,30):
            
            if(stadium_info_array[i][1] == awayTeam):
                awayLat = stadium_info_array[i][2]
                awayLong = stadium_info_array[i][3]
                
            if(stadium_info_array[i][1] == homeTeam):
                homeLat = stadium_info_array[i][2]
                homeLong = stadium_info_array[i][3]
            
        distance = find_distance(awayLat, awayLong, homeLat, homeLong)
                
        dist_matrix = update_distance_matrix(dist_matrix, team_ID_lookup(awayTeam), team_ID_lookup(homeTeam), distance)
        
    return distance

In [11]:
def OBP_calc(H, BB, HBP, AB, SF):
    return ((H+BB+HBP)/(BB+HBP+AB+SF))

def SLG_calc(H, twoB, threeB, HR, AB):
    return (((H-(twoB+threeB+HR))+(twoB*2)+(threeB*3)+(HR*4))/AB)
    
def ISO_calc(SLG, BA):
    return SLG-BA
    
def FIP_calc(HR, HBP, BB, K, IP):
    return ((((HR*13)+(3*(HBP+BB))-(2*K))/IP)+3.2)

In [12]:
#This just checks the distance matrix to make sure it is updating correctly

def check_distance_matrix():
    for i in range(0,30):
        print(dist_matrix[i])

In [29]:
#This gets us the data that we need for the analysis
dfGameData = dfGameInfo[['awayTeam','homeTeam','awayScore','homeScore','lengthInOuts','attendance','awayAB','awayH',
                                  'away2B','away3B','awayHR','awayRBI','awaySH','awaySF','awayHBP','awayBB','awaySO',
                                 'awayER','homeAB','homeH','home2B','home3B','homeHR','homeRBI','homeSH','homeSF','homeHBP',
                                  'homeBB','homeSO','homeER','timeOfDay']].copy()


#Here we add the columns that we need including distance, time zones, OBP, SLG, etc.
dfGameData['distance'] = -1
dfGameData['timeZoneChange'] = -1
dfGameData['away Batting R/9'] = -1.0
dfGameData['away Batting H/9'] = -1.0
dfGameData['away Batting BB/9'] = -1.0
dfGameData['away Batting K/9'] = -1.0
dfGameData['awayBA'] = -1.0
dfGameData['awayOBP'] = -1.0
dfGameData['awaySLG'] = -1.0
dfGameData['awayISO'] = -1.0
dfGameData['away Pitching H/9'] = -1.0
dfGameData['away Pitching BB/9'] = -1.0
dfGameData['away Pitching K/9'] = -1.0
dfGameData['awayERA'] = -1.0
dfGameData['awayFIP'] = -1.0


#Loop through an calculate the values 
#WARNING: Not pretty
for i, row in dfGameInfo.iterrows():
    
    #The number of innings the away team hit and pitched
    away_innings_hit = math.ceil(row['lengthInOuts']/6.0)
    away_innings_pitch = math.floor(row['lengthInOuts']/6.0)
    
    #distance
    distance_val = get_distance(str(row['awayTeam']), str(row['homeTeam']))
    dfGameData.set_value(i, 'distance', distance_val)
    
    #time zone change
    t_z = find_time_zone_difference(str(row['awayTeam']), str(row['homeTeam']))
    dfGameData.set_value(i, 'timeZoneChange', t_z)
    
    #away batting R/9
    bat_run_per_nine = row['awayScore']*(9.0/away_innings_hit)
    dfGameData.set_value(i, 'away Batting R/9', bat_run_per_nine)
    
    #away batting H/9
    bat_hit_per_nine = row['awayH']*(9.0/away_innings_hit)
    dfGameData.set_value(i, 'away Batting H/9', bat_hit_per_nine)
    
    #away batting BB/9
    bat_walk_per_nine = row['awayBB']*(9.0/away_innings_hit)
    dfGameData.set_value(i, 'away Batting BB/9', bat_walk_per_nine)
    
    #away batting K/9
    bat_k_per_nine = row['awaySO']*(9.0/away_innings_hit)
    dfGameData.set_value(i, 'away Batting K/9', bat_k_per_nine)
    
    #away BA
    BA = row['awayH']/row['awayAB']
    dfGameData.set_value(i, 'awayBA', BA)
    
    #away OBP
    OBP = OBP_calc(row['awayH'], row['awayBB'], row['awayHBP'], row['awayAB'], row['awaySF'])
    dfGameData.set_value(i, 'awayOBP', OBP)
    
    #away SLG
    SLG = SLG_calc(row['awayH'], row['away2B'], row['away3B'], row['awayHR'], row['awayAB'])
    dfGameData.set_value(i, 'awaySLG', SLG)
    
    #away ISO
    ISO = ISO_calc(SLG_calc(row['awayH'], row['away2B'], row['away3B'], row['awayHR'], row['awayAB']), row['awayH']/row['awayAB'])
    dfGameData.set_value(i, 'awayISO', ISO)
    
    #away pitching H/9
    pit_hit_per_nine = row['homeH']*(9.0/away_innings_pitch)
    dfGameData.set_value(i, 'away Pitching H/9', pit_hit_per_nine)
    
    #away pitching BB/9
    pit_walk_per_nine = row['homeBB']*(9.0/away_innings_pitch)
    dfGameData.set_value(i, 'away Pitching BB/9', pit_walk_per_nine)
    
    #away pitching K/9
    pit_k_per_nine = row['homeSO']*(9.0/away_innings_pitch)
    dfGameData.set_value(i, 'away Pitching K/9', pit_k_per_nine)
    
    #away ERA
    ERA = row['homeER']*(9.0/away_innings_pitch)
    dfGameData.set_value(i, 'awayERA', ERA)
    
    #away FIP
    FIP = FIP_calc(row['homeHR'], row['homeHBP'], row['homeBB'], row['homeSO'], away_innings_pitch)
    dfGameData.set_value(i, 'awayFIP', FIP)

In [30]:
#Create a dataframe for each distance section and time zone change
dfGameData = dfGameData.dropna()
dfGameData0_500 = dfGameData.loc[(dfGameData['distance'] < 500) & (dfGameData['distance'] >= 1)]
dfGameData500_1000 = dfGameData.loc[(dfGameData['distance'] < 1000) & (dfGameData['distance'] >= 500)]
dfGameData1000_2000 = dfGameData.loc[(dfGameData['distance'] < 2000) & (dfGameData['distance'] >= 1000)]
dfGameData2000 = dfGameData.loc[(dfGameData['distance'] < 10000) & (dfGameData['distance'] >= 2000)]
dfGameData3hourswest = dfGameData.loc[(dfGameData['timeZoneChange'] == -3) & (dfGameData['timeOfDay'] == 'N')]
dfGameData2hourswest = dfGameData.loc[dfGameData['timeZoneChange'] == -2]
dfGameData1hourswest = dfGameData.loc[dfGameData['timeZoneChange'] == -1]
dfGameData0hours = dfGameData.loc[dfGameData['timeZoneChange'] == 0]
dfGameData1hourseast = dfGameData.loc[dfGameData['timeZoneChange'] == 1]
dfGameData2hourseast = dfGameData.loc[dfGameData['timeZoneChange'] == 2]
dfGameData3hourseast = dfGameData.loc[dfGameData['timeZoneChange'] == 3]

In [15]:
dfGameData3hourseast.mean()

awayScore                 4.623359
homeScore                 4.678714
lengthInOuts             53.632110
attendance            28478.285144
awayAB                   35.065817
awayH                     9.089214
away2B                    1.779912
away3B                    0.171010
awayHR                    1.020734
awayRBI                   4.384630
awaySH                    0.328134
awaySF                    0.290280
awayHBP                   0.324329
awayBB                    3.338406
awaySO                    6.937417
awayER                    4.276013
homeAB                   33.484307
homeH                     8.835648
home2B                    1.755944
home3B                    0.182994
homeHR                    1.048887
homeRBI                   4.451969
homeSH                    0.310443
homeSF                    0.272779
homeHBP                   0.325090
homeBB                    3.482975
homeSO                    6.628495
homeER                    4.226555
distance            

In [16]:
print ("p values at < 500 miles traveled")
print ("--------------------------------")
print ("< 500 miles distance count: \t\t\t\t",len(dfGameData0_500.index))
print ("< 500 miles distance away bat. R/9 p value: \t\t",(stats.ttest_ind(dfGameData0_500['away Batting R/9'], dfGameData['away Batting R/9'])[1]))
print ("< 500 miles distance away bat. H/9 p value: \t\t",(stats.ttest_ind(dfGameData0_500['away Batting H/9'], dfGameData['away Batting H/9'])[1]))
print ("< 500 miles distance away bat. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData0_500['away Batting BB/9'], dfGameData['away Batting BB/9'])[1]))
print ("< 500 miles distance away bat.  K/9 p value: \t\t",(stats.ttest_ind(dfGameData0_500['away Batting K/9'], dfGameData['away Batting K/9'])[1]))
print ("< 500 miles distance away bat. BA p value: \t\t",(stats.ttest_ind(dfGameData0_500['awayBA'], dfGameData['awayBA'])[1]))
print ("< 500 miles distance away bat. OBP p value: \t\t",(stats.ttest_ind(dfGameData0_500['awayOBP'], dfGameData['awayOBP'])[1]))
print ("< 500 miles distance away bat. SLG p value: \t\t",(stats.ttest_ind(dfGameData0_500['awaySLG'], dfGameData['awaySLG'])[1]))
print ("< 500 miles distance away bat. ISO p value: \t\t",(stats.ttest_ind(dfGameData0_500['awayISO'], dfGameData['awayISO'])[1]))
print ("< 500 miles distance away pit. H/9 p value: \t\t",(stats.ttest_ind(dfGameData0_500['away Pitching H/9'], dfGameData['away Pitching H/9'])[1]))
print ("< 500 miles distance away pit. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData0_500['away Pitching BB/9'], dfGameData['away Pitching BB/9'])[1]))
print ("< 500 miles distance away pit. K/9 p value: \t\t",(stats.ttest_ind(dfGameData0_500['away Pitching K/9'], dfGameData['away Pitching K/9'])[1]))
print ("< 500 miles distance away pit. ERA p value: \t\t",(stats.ttest_ind(dfGameData0_500['awayERA'], dfGameData['awayERA'])[1]))
print ("< 500 miles distance away pit. FIP p value: \t\t",(stats.ttest_ind(dfGameData0_500['awayFIP'], dfGameData['awayFIP'])[1]))

p values at < 500 miles traveled
--------------------------------
< 500 miles distance count: 				 18932
< 500 miles distance away bat. R/9 p value: 		 0.980672256052
< 500 miles distance away bat. H/9 p value: 		 0.576710316215
< 500 miles distance away bat. BB/9 p value: 		 0.862544599146
< 500 miles distance away bat.  K/9 p value: 		 0.801893360973
< 500 miles distance away bat. BA p value: 		 0.568041754913
< 500 miles distance away bat. OBP p value: 		 0.723249337387
< 500 miles distance away bat. SLG p value: 		 0.459430706151
< 500 miles distance away bat. ISO p value: 		 0.111501211157
< 500 miles distance away pit. H/9 p value: 		 0.0374802685207
< 500 miles distance away pit. BB/9 p value: 		 0.0972540780333
< 500 miles distance away pit. K/9 p value: 		 0.103370376926
< 500 miles distance away pit. ERA p value: 		 0.655474054146
< 500 miles distance away pit. FIP p value: 		 0.213544448418


In [17]:
print ("p values at 500 - 1000 miles traveled")
print ("-------------------------------------")
print ("500 - 1000 miles distance count: \t\t\t\t",len(dfGameData500_1000.index))
print ("500 - 1000 miles distance away bat. R/9 p value: \t\t",(stats.ttest_ind(dfGameData500_1000['away Batting R/9'], dfGameData['away Batting R/9'])[1]))
print ("500 - 1000 miles distance away bat. H/9 p value: \t\t",(stats.ttest_ind(dfGameData500_1000['away Batting H/9'], dfGameData['away Batting H/9'])[1]))
print ("500 - 1000 miles distance away bat. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData500_1000['away Batting BB/9'], dfGameData['away Batting BB/9'])[1]))
print ("500 - 1000 miles distance away bat.  K/9 p value: \t\t",(stats.ttest_ind(dfGameData500_1000['away Batting K/9'], dfGameData['away Batting K/9'])[1]))
print ("500 - 1000 miles distance away bat. BA p value: \t\t",(stats.ttest_ind(dfGameData500_1000['awayBA'], dfGameData['awayBA'])[1]))
print ("500 - 1000 miles distance away bat. OBP p value: \t\t",(stats.ttest_ind(dfGameData500_1000['awayOBP'], dfGameData['awayOBP'])[1]))
print ("500 - 1000 miles distance away bat. SLG p value: \t\t",(stats.ttest_ind(dfGameData500_1000['awaySLG'], dfGameData['awaySLG'])[1]))
print ("500 - 1000 miles distance away bat. ISO p value: \t\t",(stats.ttest_ind(dfGameData500_1000['awayISO'], dfGameData['awayISO'])[1]))
print ("500 - 1000 miles distance away pit. H/9 p value: \t\t",(stats.ttest_ind(dfGameData500_1000['away Pitching H/9'], dfGameData['away Pitching H/9'])[1]))
print ("500 - 1000 miles distance away pit. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData500_1000['away Pitching BB/9'], dfGameData['away Pitching BB/9'])[1]))
print ("500 - 1000 miles distance away pit. K/9 p value: \t\t",(stats.ttest_ind(dfGameData500_1000['away Pitching K/9'], dfGameData['away Pitching K/9'])[1]))
print ("500 - 1000 miles distance away pit. ERA p value: \t\t",(stats.ttest_ind(dfGameData500_1000['awayERA'], dfGameData['awayERA'])[1]))
print ("500 - 1000 miles distance away pit. FIP p value: \t\t",(stats.ttest_ind(dfGameData500_1000['awayFIP'], dfGameData['awayFIP'])[1]))

p values at 500 - 1000 miles traveled
-------------------------------------
500 - 1000 miles distance count: 				 16678
500 - 1000 miles distance away bat. R/9 p value: 		 0.185479190758
500 - 1000 miles distance away bat. H/9 p value: 		 0.00286647725268
500 - 1000 miles distance away bat. BB/9 p value: 		 0.133418048413
500 - 1000 miles distance away bat.  K/9 p value: 		 0.0560950039594
500 - 1000 miles distance away bat. BA p value: 		 0.00118049511971
500 - 1000 miles distance away bat. OBP p value: 		 0.0774720596519
500 - 1000 miles distance away bat. SLG p value: 		 0.0254859519857
500 - 1000 miles distance away bat. ISO p value: 		 0.394182190275
500 - 1000 miles distance away pit. H/9 p value: 		 0.000757483582988
500 - 1000 miles distance away pit. BB/9 p value: 		 0.15430517325
500 - 1000 miles distance away pit. K/9 p value: 		 0.00132333631569
500 - 1000 miles distance away pit. ERA p value: 		 0.22130645356
500 - 1000 miles distance away pit. FIP p value: 		 0.3477728088

In [18]:
print ("p values at 1000 - 2000 miles traveled")
print ("--------------------------------------")
print ("1000 - 2000 miles distance count: \t\t\t\t",len(dfGameData1000_2000.index))
print ("1000 - 2000 miles distance away bat. R/9 p value: \t\t",(stats.ttest_ind(dfGameData1000_2000['away Batting R/9'], dfGameData['away Batting R/9'])[1]))
print ("1000 - 2000 miles distance away bat. H/9 p value: \t\t",(stats.ttest_ind(dfGameData1000_2000['away Batting H/9'], dfGameData['away Batting H/9'])[1]))
print ("1000 - 2000 miles distance away bat. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData1000_2000['away Batting BB/9'], dfGameData['away Batting BB/9'])[1]))
print ("1000 - 2000 miles distance away bat.  K/9 p value: \t\t",(stats.ttest_ind(dfGameData1000_2000['away Batting K/9'], dfGameData['away Batting K/9'])[1]))
print ("1000 - 2000 miles distance away bat. BA p value: \t\t",(stats.ttest_ind(dfGameData1000_2000['awayBA'], dfGameData['awayBA'])[1]))
print ("1000 - 2000 miles distance away bat. OBP p value: \t\t",(stats.ttest_ind(dfGameData1000_2000['awayOBP'], dfGameData['awayOBP'])[1]))
print ("1000 - 2000 miles distance away bat. SLG p value: \t\t",(stats.ttest_ind(dfGameData1000_2000['awaySLG'], dfGameData['awaySLG'])[1]))
print ("1000 - 2000 miles distance away bat. ISO p value: \t\t",(stats.ttest_ind(dfGameData1000_2000['awayISO'], dfGameData['awayISO'])[1]))
print ("1000 - 2000 miles distance away pit. H/9 p value: \t\t",(stats.ttest_ind(dfGameData1000_2000['away Pitching H/9'], dfGameData['away Pitching H/9'])[1]))
print ("1000 - 2000 miles distance away pit. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData1000_2000['away Pitching BB/9'], dfGameData['away Pitching BB/9'])[1]))
print ("1000 - 2000 miles distance away pit. K/9 p value: \t\t",(stats.ttest_ind(dfGameData1000_2000['away Pitching K/9'], dfGameData['away Pitching K/9'])[1]))
print ("1000 - 2000 miles distance away pit. ERA p value: \t\t",(stats.ttest_ind(dfGameData1000_2000['awayERA'], dfGameData['awayERA'])[1]))
print ("1000 - 2000 miles distance away pit. FIP p value: \t\t",(stats.ttest_ind(dfGameData1000_2000['awayFIP'], dfGameData['awayFIP'])[1]))

p values at 1000 - 2000 miles traveled
--------------------------------------
1000 - 2000 miles distance count: 				 18130
1000 - 2000 miles distance away bat. R/9 p value: 		 0.330036223184
1000 - 2000 miles distance away bat. H/9 p value: 		 0.955657648138
1000 - 2000 miles distance away bat. BB/9 p value: 		 0.884505547987
1000 - 2000 miles distance away bat.  K/9 p value: 		 0.00866311079575
1000 - 2000 miles distance away bat. BA p value: 		 0.786985471057
1000 - 2000 miles distance away bat. OBP p value: 		 0.880162705751
1000 - 2000 miles distance away bat. SLG p value: 		 0.980203480746
1000 - 2000 miles distance away bat. ISO p value: 		 0.799237224651
1000 - 2000 miles distance away pit. H/9 p value: 		 0.100542808259
1000 - 2000 miles distance away pit. BB/9 p value: 		 0.190263435417
1000 - 2000 miles distance away pit. K/9 p value: 		 0.201702878729
1000 - 2000 miles distance away pit. ERA p value: 		 0.335457785051
1000 - 2000 miles distance away pit. FIP p value: 		 0.87

In [19]:
print ("p values at > 2000 miles traveled")
print ("---------------------------------")
print ("> 2000 miles distance count: \t\t\t\t",len(dfGameData2000.index))
print ("> 2000 miles distance away bat. R/9 p value: \t\t",(stats.ttest_ind(dfGameData2000['away Batting R/9'], dfGameData['away Batting R/9'])[1]))
print ("> 2000 miles distance away bat. H/9 p value: \t\t",(stats.ttest_ind(dfGameData2000['away Batting H/9'], dfGameData['away Batting H/9'])[1]))
print ("> 2000 miles distance away bat. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData2000['away Batting BB/9'], dfGameData['away Batting BB/9'])[1]))
print ("> 2000 miles distance away bat.  K/9 p value: \t\t",(stats.ttest_ind(dfGameData2000['away Batting K/9'], dfGameData['away Batting K/9'])[1]))
print ("> 2000 miles distance away bat. BA p value: \t\t",(stats.ttest_ind(dfGameData2000['awayBA'], dfGameData['awayBA'])[1]))
print ("> 2000 miles distance away bat. OBP p value: \t\t",(stats.ttest_ind(dfGameData2000['awayOBP'], dfGameData['awayOBP'])[1]))
print ("> 2000 miles distance away bat. SLG p value: \t\t",(stats.ttest_ind(dfGameData2000['awaySLG'], dfGameData['awaySLG'])[1]))
print ("> 2000 miles distance away bat. ISO p value: \t\t",(stats.ttest_ind(dfGameData2000['awayISO'], dfGameData['awayISO'])[1]))
print ("> 2000 miles distance away pit. H/9 p value: \t\t",(stats.ttest_ind(dfGameData2000['away Pitching H/9'], dfGameData['away Pitching H/9'])[1]))
print ("> 2000 miles distance away pit. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData2000['away Pitching BB/9'], dfGameData['away Pitching BB/9'])[1]))
print ("> 2000 miles distance away pit. K/9 p value: \t\t",(stats.ttest_ind(dfGameData2000['away Pitching K/9'], dfGameData['away Pitching K/9'])[1]))
print ("> 2000 miles distance away pit. ERA p value: \t\t",(stats.ttest_ind(dfGameData2000['awayERA'], dfGameData['awayERA'])[1]))
print ("> 2000 miles distance away pit. FIP p value: \t\t",(stats.ttest_ind(dfGameData2000['awayFIP'], dfGameData['awayFIP'])[1]))

p values at > 2000 miles traveled
---------------------------------
> 2000 miles distance count: 				 10393
> 2000 miles distance away bat. R/9 p value: 		 0.00163404230491
> 2000 miles distance away bat. H/9 p value: 		 0.00125824731766
> 2000 miles distance away bat. BB/9 p value: 		 0.0435057178893
> 2000 miles distance away bat.  K/9 p value: 		 0.139710397404
> 2000 miles distance away bat. BA p value: 		 0.00197548823328
> 2000 miles distance away bat. OBP p value: 		 0.105660994075
> 2000 miles distance away bat. SLG p value: 		 5.14709944417e-05
> 2000 miles distance away bat. ISO p value: 		 0.000166895677225
> 2000 miles distance away pit. H/9 p value: 		 0.000150569895628
> 2000 miles distance away pit. BB/9 p value: 		 0.0160057259741
> 2000 miles distance away pit. K/9 p value: 		 0.897956385031
> 2000 miles distance away pit. ERA p value: 		 0.000307512575617
> 2000 miles distance away pit. FIP p value: 		 0.751365177127


In [32]:
print ("p values at 3 time zones traveled west")
print ("--------------------------------------")
print ("3 hours west traveled count: \t\t\t\t",len(dfGameData3hourswest.index))
print ("3 hours west traveled away bat. R/9 p value: \t\t",(stats.ttest_ind(dfGameData3hourswest['away Batting R/9'], dfGameData['away Batting R/9'])[1]))
print ("3 hours west traveled away bat. H/9 p value: \t\t",(stats.ttest_ind(dfGameData3hourswest['away Batting H/9'], dfGameData['away Batting H/9'])[1]))
print ("3 hours west traveled away bat. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData3hourswest['away Batting BB/9'], dfGameData['away Batting BB/9'])[1]))
print ("3 hours west traveled away bat.  K/9 p value: \t\t",(stats.ttest_ind(dfGameData3hourswest['away Batting K/9'], dfGameData['away Batting K/9'])[1]))
print ("3 hours west traveled away bat. BA p value: \t\t",(stats.ttest_ind(dfGameData3hourswest['awayBA'], dfGameData['awayBA'])[1]))
print ("3 hours west traveled away bat. OBP p value: \t\t",(stats.ttest_ind(dfGameData3hourswest['awayOBP'], dfGameData['awayOBP'])[1]))
print ("3 hours west traveled away bat. SLG p value: \t\t",(stats.ttest_ind(dfGameData3hourswest['awaySLG'], dfGameData['awaySLG'])[1]))
print ("3 hours west traveled away bat. ISO p value: \t\t",(stats.ttest_ind(dfGameData3hourswest['awayISO'], dfGameData['awayISO'])[1]))
print ("3 hours west traveled away pit. H/9 p value: \t\t",(stats.ttest_ind(dfGameData3hourswest['away Pitching H/9'], dfGameData['away Pitching H/9'])[1]))
print ("3 hours west traveled away pit. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData3hourswest['away Pitching BB/9'], dfGameData['away Pitching BB/9'])[1]))
print ("3 hours west traveled away pit. K/9 p value: \t\t",(stats.ttest_ind(dfGameData3hourswest['away Pitching K/9'], dfGameData['away Pitching K/9'])[1]))
print ("3 hours west traveled away pit. ERA p value: \t\t",(stats.ttest_ind(dfGameData3hourswest['awayERA'], dfGameData['awayERA'])[1]))
print ("3 hours west traveled away pit. FIP p value: \t\t",(stats.ttest_ind(dfGameData3hourswest['awayFIP'], dfGameData['awayFIP'])[1]))

p values at 3 time zones traveled west
--------------------------------------
3 hours west traveled count: 				 3654
3 hours west traveled away bat. R/9 p value: 		 1.43447933372e-08
3 hours west traveled away bat. H/9 p value: 		 1.31751069967e-08
3 hours west traveled away bat. BB/9 p value: 		 0.326142718295
3 hours west traveled away bat.  K/9 p value: 		 0.00435264172084
3 hours west traveled away bat. BA p value: 		 1.98138808886e-08
3 hours west traveled away bat. OBP p value: 		 5.67686418023e-08
3 hours west traveled away bat. SLG p value: 		 7.06958911956e-08
3 hours west traveled away bat. ISO p value: 		 0.000136521297936
3 hours west traveled away pit. H/9 p value: 		 6.16479805865e-07
3 hours west traveled away pit. BB/9 p value: 		 0.133547549182
3 hours west traveled away pit. K/9 p value: 		 0.255498270022
3 hours west traveled away pit. ERA p value: 		 6.82784311062e-09
3 hours west traveled away pit. FIP p value: 		 1.93612667264e-06


In [21]:
print ("p values at 2 time zones traveled west")
print ("--------------------------------------")
print ("2 hours west traveled count: \t\t\t\t",len(dfGameData2hourswest.index))
print ("2 hours west traveled away bat. R/9 p value: \t\t",(stats.ttest_ind(dfGameData2hourswest['away Batting R/9'], dfGameData['away Batting R/9'])[1]))
print ("2 hours west traveled away bat. H/9 p value: \t\t",(stats.ttest_ind(dfGameData2hourswest['away Batting H/9'], dfGameData['away Batting H/9'])[1]))
print ("2 hours west traveled away bat. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData2hourswest['away Batting BB/9'], dfGameData['away Batting BB/9'])[1]))
print ("2 hours west traveled away bat.  K/9 p value: \t\t",(stats.ttest_ind(dfGameData2hourswest['away Batting K/9'], dfGameData['away Batting K/9'])[1]))
print ("2 hours west traveled away bat. BA p value: \t\t",(stats.ttest_ind(dfGameData2hourswest['awayBA'], dfGameData['awayBA'])[1]))
print ("2 hours west traveled away bat. OBP p value: \t\t",(stats.ttest_ind(dfGameData2hourswest['awayOBP'], dfGameData['awayOBP'])[1]))
print ("2 hours west traveled away bat. SLG p value: \t\t",(stats.ttest_ind(dfGameData2hourswest['awaySLG'], dfGameData['awaySLG'])[1]))
print ("2 hours west traveled away bat. ISO p value: \t\t",(stats.ttest_ind(dfGameData2hourswest['awayISO'], dfGameData['awayISO'])[1]))
print ("2 hours west traveled away pit. H/9 p value: \t\t",(stats.ttest_ind(dfGameData2hourswest['away Pitching H/9'], dfGameData['away Pitching H/9'])[1]))
print ("2 hours west traveled away pit. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData2hourswest['away Pitching BB/9'], dfGameData['away Pitching BB/9'])[1]))
print ("2 hours west traveled away pit. K/9 p value: \t\t",(stats.ttest_ind(dfGameData2hourswest['away Pitching K/9'], dfGameData['away Pitching K/9'])[1]))
print ("2 hours west traveled away pit. ERA p value: \t\t",(stats.ttest_ind(dfGameData2hourswest['awayERA'], dfGameData['awayERA'])[1]))
print ("2 hours west traveled away pit. FIP p value: \t\t",(stats.ttest_ind(dfGameData2hourswest['awayFIP'], dfGameData['awayFIP'])[1]))

p values at 2 time zones traveled west
--------------------------------------
2 hours west traveled count: 				 4336
2 hours west traveled away bat. R/9 p value: 		 0.710983526427
2 hours west traveled away bat. H/9 p value: 		 0.373253952446
2 hours west traveled away bat. BB/9 p value: 		 0.432304670239
2 hours west traveled away bat.  K/9 p value: 		 0.724716016502
2 hours west traveled away bat. BA p value: 		 0.298142610474
2 hours west traveled away bat. OBP p value: 		 0.36562324842
2 hours west traveled away bat. SLG p value: 		 0.255393150338
2 hours west traveled away bat. ISO p value: 		 0.356710860396
2 hours west traveled away pit. H/9 p value: 		 0.0285682504796
2 hours west traveled away pit. BB/9 p value: 		 0.189646500624
2 hours west traveled away pit. K/9 p value: 		 0.00124456917029
2 hours west traveled away pit. ERA p value: 		 0.735659998814
2 hours west traveled away pit. FIP p value: 		 0.534853348196


In [22]:
print ("p values at 1 time zone traveled west")
print ("-------------------------------------")
print ("1 hour west traveled count: \t\t\t\t",len(dfGameData1hourswest.index))
print ("1 hour west traveled away bat. R/9 p value: \t\t",(stats.ttest_ind(dfGameData1hourswest['away Batting R/9'], dfGameData['away Batting R/9'])[1]))
print ("1 hour west traveled away bat. H/9 p value: \t\t",(stats.ttest_ind(dfGameData1hourswest['away Batting H/9'], dfGameData['away Batting H/9'])[1]))
print ("1 hour west traveled away bat. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData1hourswest['away Batting BB/9'], dfGameData['away Batting BB/9'])[1]))
print ("1 hour west traveled away bat.  K/9 p value: \t\t",(stats.ttest_ind(dfGameData1hourswest['away Batting K/9'], dfGameData['away Batting K/9'])[1]))
print ("1 hour west traveled away bat. BA p value: \t\t",(stats.ttest_ind(dfGameData1hourswest['awayBA'], dfGameData['awayBA'])[1]))
print ("1 hour west traveled away bat. OBP p value: \t\t",(stats.ttest_ind(dfGameData1hourswest['awayOBP'], dfGameData['awayOBP'])[1]))
print ("1 hour west traveled away bat. SLG p value: \t\t",(stats.ttest_ind(dfGameData1hourswest['awaySLG'], dfGameData['awaySLG'])[1]))
print ("1 hour west traveled away bat. ISO p value: \t\t",(stats.ttest_ind(dfGameData1hourswest['awayISO'], dfGameData['awayISO'])[1]))
print ("1 hour west traveled away pit. H/9 p value: \t\t",(stats.ttest_ind(dfGameData1hourswest['away Pitching H/9'], dfGameData['away Pitching H/9'])[1]))
print ("1 hour west traveled away pit. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData1hourswest['away Pitching BB/9'], dfGameData['away Pitching BB/9'])[1]))
print ("1 hour west traveled away pit. K/9 p value: \t\t",(stats.ttest_ind(dfGameData1hourswest['away Pitching K/9'], dfGameData['away Pitching K/9'])[1]))
print ("1 hour west traveled away pit. ERA p value: \t\t",(stats.ttest_ind(dfGameData1hourswest['awayERA'], dfGameData['awayERA'])[1]))
print ("1 hour west traveled away pit. FIP p value: \t\t",(stats.ttest_ind(dfGameData1hourswest['awayFIP'], dfGameData['awayFIP'])[1]))

p values at 1 time zone traveled west
-------------------------------------
1 hour west traveled count: 				 9120
1 hour west traveled away bat. R/9 p value: 		 0.000662089272735
1 hour west traveled away bat. H/9 p value: 		 3.78219679099e-05
1 hour west traveled away bat. BB/9 p value: 		 0.954682068282
1 hour west traveled away bat.  K/9 p value: 		 0.342858177461
1 hour west traveled away bat. BA p value: 		 3.38636122391e-05
1 hour west traveled away bat. OBP p value: 		 0.000327625646531
1 hour west traveled away bat. SLG p value: 		 3.15691618176e-07
1 hour west traveled away bat. ISO p value: 		 4.9623289036e-06
1 hour west traveled away pit. H/9 p value: 		 7.536190608e-06
1 hour west traveled away pit. BB/9 p value: 		 0.174681208213
1 hour west traveled away pit. K/9 p value: 		 4.55873483064e-09
1 hour west traveled away pit. ERA p value: 		 0.000118856427867
1 hour west traveled away pit. FIP p value: 		 0.959611323892


In [23]:
print ("p values at 0 time zone traveled")
print ("--------------------------------")
print ("0 hours traveled count: \t\t\t\t",len(dfGameData0hours.index))
print ("0 hours traveled away bat. R/9 p value: \t\t",(stats.ttest_ind(dfGameData0hours['away Batting R/9'], dfGameData['away Batting R/9'])[1]))
print ("0 hours traveled away bat. H/9 p value: \t\t",(stats.ttest_ind(dfGameData0hours['away Batting H/9'], dfGameData['away Batting H/9'])[1]))
print ("0 hours traveled away bat. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData0hours['away Batting BB/9'], dfGameData['away Batting BB/9'])[1]))
print ("0 hours traveled away bat.  K/9 p value: \t\t",(stats.ttest_ind(dfGameData0hours['away Batting K/9'], dfGameData['away Batting K/9'])[1]))
print ("0 hours traveled away bat. BA p value: \t\t\t",(stats.ttest_ind(dfGameData0hours['awayBA'], dfGameData['awayBA'])[1]))
print ("0 hours traveled away bat. OBP p value: \t\t",(stats.ttest_ind(dfGameData0hours['awayOBP'], dfGameData['awayOBP'])[1]))
print ("0 hours traveled away bat. SLG p value: \t\t",(stats.ttest_ind(dfGameData0hours['awaySLG'], dfGameData['awaySLG'])[1]))
print ("0 hours traveled away bat. ISO p value: \t\t",(stats.ttest_ind(dfGameData0hours['awayISO'], dfGameData['awayISO'])[1]))
print ("0 hours traveled away pit. H/9 p value: \t\t",(stats.ttest_ind(dfGameData0hours['away Pitching H/9'], dfGameData['away Pitching H/9'])[1]))
print ("0 hours traveled away pit. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData0hours['away Pitching BB/9'], dfGameData['away Pitching BB/9'])[1]))
print ("0 hours traveled away pit. K/9 p value: \t\t",(stats.ttest_ind(dfGameData0hours['away Pitching K/9'], dfGameData['away Pitching K/9'])[1]))
print ("0 hours traveled away pit. ERA p value: \t\t",(stats.ttest_ind(dfGameData0hours['awayERA'], dfGameData['awayERA'])[1]))
print ("0 hours traveled away pit. FIP p value: \t\t",(stats.ttest_ind(dfGameData0hours['awayFIP'], dfGameData['awayFIP'])[1]))

p values at 0 time zone traveled
--------------------------------
0 hours traveled count: 				 23350
0 hours traveled away bat. R/9 p value: 		 0.499167713829
0 hours traveled away bat. H/9 p value: 		 0.141159531232
0 hours traveled away bat. BB/9 p value: 		 0.21170976952
0 hours traveled away bat.  K/9 p value: 		 0.00108180535375
0 hours traveled away bat. BA p value: 			 0.0901809969495
0 hours traveled away bat. OBP p value: 		 0.519373715371
0 hours traveled away bat. SLG p value: 		 0.798741607865
0 hours traveled away bat. ISO p value: 		 0.337753334211
0 hours traveled away pit. H/9 p value: 		 0.00720877520815
0 hours traveled away pit. BB/9 p value: 		 0.440713609926
0 hours traveled away pit. K/9 p value: 		 1.14799238911e-07
0 hours traveled away pit. ERA p value: 		 0.684927249168
0 hours traveled away pit. FIP p value: 		 0.375628519613


In [24]:
print ("p values at 1 time zone traveled east")
print ("-------------------------------------")
print ("1 hour east traveled count: \t\t\t\t",len(dfGameData1hourseast.index))
print ("1 hour east traveled away bat. R/9 p value: \t\t",(stats.ttest_ind(dfGameData1hourseast['away Batting R/9'], dfGameData['away Batting R/9'])[1]))
print ("1 hour east traveled away bat. H/9 p value: \t\t",(stats.ttest_ind(dfGameData1hourseast['away Batting H/9'], dfGameData['away Batting H/9'])[1]))
print ("1 hour east traveled away bat. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData1hourseast['away Batting BB/9'], dfGameData['away Batting BB/9'])[1]))
print ("1 hour east traveled away bat.  K/9 p value: \t\t",(stats.ttest_ind(dfGameData1hourseast['away Batting K/9'], dfGameData['away Batting K/9'])[1]))
print ("1 hour east traveled away bat. BA p value: \t\t",(stats.ttest_ind(dfGameData1hourseast['awayBA'], dfGameData['awayBA'])[1]))
print ("1 hour east traveled away bat. OBP p value: \t\t",(stats.ttest_ind(dfGameData1hourseast['awayOBP'], dfGameData['awayOBP'])[1]))
print ("1 hour east traveled away bat. SLG p value: \t\t",(stats.ttest_ind(dfGameData1hourseast['awaySLG'], dfGameData['awaySLG'])[1]))
print ("1 hour east traveled away bat. ISO p value: \t\t",(stats.ttest_ind(dfGameData1hourseast['awayISO'], dfGameData['awayISO'])[1]))
print ("1 hour east traveled away pit. H/9 p value: \t\t",(stats.ttest_ind(dfGameData1hourseast['away Pitching H/9'], dfGameData['away Pitching H/9'])[1]))
print ("1 hour east traveled away pit. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData1hourseast['away Pitching BB/9'], dfGameData['away Pitching BB/9'])[1]))
print ("1 hour east traveled away pit. K/9 p value: \t\t",(stats.ttest_ind(dfGameData1hourseast['away Pitching K/9'], dfGameData['away Pitching K/9'])[1]))
print ("1 hour east traveled away pit. ERA p value: \t\t",(stats.ttest_ind(dfGameData1hourseast['awayERA'], dfGameData['awayERA'])[1]))
print ("1 hour east traveled away pit. FIP p value: \t\t",(stats.ttest_ind(dfGameData1hourseast['awayFIP'], dfGameData['awayFIP'])[1]))

p values at 1 time zone traveled east
-------------------------------------
1 hour east traveled count: 				 9123
1 hour east traveled away bat. R/9 p value: 		 0.0126112857042
1 hour east traveled away bat. H/9 p value: 		 5.6788984719e-05
1 hour east traveled away bat. BB/9 p value: 		 0.00246119725597
1 hour east traveled away bat.  K/9 p value: 		 0.021250608384
1 hour east traveled away bat. BA p value: 		 2.79421757829e-05
1 hour east traveled away bat. OBP p value: 		 0.0866196446082
1 hour east traveled away bat. SLG p value: 		 0.00294677497041
1 hour east traveled away bat. ISO p value: 		 0.217313559379
1 hour east traveled away pit. H/9 p value: 		 2.24745445507e-07
1 hour east traveled away pit. BB/9 p value: 		 0.467773832416
1 hour east traveled away pit. K/9 p value: 		 0.0294454760491
1 hour east traveled away pit. ERA p value: 		 0.0145442580556
1 hour east traveled away pit. FIP p value: 		 2.03746547151e-09


In [25]:
print ("p values at 2 time zones traveled east")
print ("--------------------------------------")
print ("2 hours east traveled count: \t\t\t\t",len(dfGameData2hourseast.index))
print ("2 hours east traveled away bat. R/9 p value: \t\t",(stats.ttest_ind(dfGameData2hourseast['away Batting R/9'], dfGameData['away Batting R/9'])[1]))
print ("2 hours east traveled away bat. H/9 p value: \t\t",(stats.ttest_ind(dfGameData2hourseast['away Batting H/9'], dfGameData['away Batting H/9'])[1]))
print ("2 hours east traveled away bat. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData2hourseast['away Batting BB/9'], dfGameData['away Batting BB/9'])[1]))
print ("2 hours east traveled away bat.  K/9 p value: \t\t",(stats.ttest_ind(dfGameData2hourseast['away Batting K/9'], dfGameData['away Batting K/9'])[1]))
print ("2 hours east traveled away bat. BA p value: \t\t",(stats.ttest_ind(dfGameData2hourseast['awayBA'], dfGameData['awayBA'])[1]))
print ("2 hours east traveled away bat. OBP p value: \t\t",(stats.ttest_ind(dfGameData2hourseast['awayOBP'], dfGameData['awayOBP'])[1]))
print ("2 hours east traveled away bat. SLG p value: \t\t",(stats.ttest_ind(dfGameData2hourseast['awaySLG'], dfGameData['awaySLG'])[1]))
print ("2 hours east traveled away bat. ISO p value: \t\t",(stats.ttest_ind(dfGameData2hourseast['awayISO'], dfGameData['awayISO'])[1]))
print ("2 hours east traveled away pit. H/9 p value: \t\t",(stats.ttest_ind(dfGameData2hourseast['away Pitching H/9'], dfGameData['away Pitching H/9'])[1]))
print ("2 hours east traveled away pit. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData2hourseast['away Pitching BB/9'], dfGameData['away Pitching BB/9'])[1]))
print ("2 hours east traveled away pit. K/9 p value: \t\t",(stats.ttest_ind(dfGameData2hourseast['away Pitching K/9'], dfGameData['away Pitching K/9'])[1]))
print ("2 hours east traveled away pit. ERA p value: \t\t",(stats.ttest_ind(dfGameData2hourseast['awayERA'], dfGameData['awayERA'])[1]))
print ("2 hours east traveled away pit. FIP p value: \t\t",(stats.ttest_ind(dfGameData2hourseast['awayFIP'], dfGameData['awayFIP'])[1]))

p values at 2 time zones traveled east
--------------------------------------
2 hours east traveled count: 				 4404
2 hours east traveled away bat. R/9 p value: 		 0.0743751443572
2 hours east traveled away bat. H/9 p value: 		 0.505773323758
2 hours east traveled away bat. BB/9 p value: 		 0.284731823491
2 hours east traveled away bat.  K/9 p value: 		 0.0920572636557
2 hours east traveled away bat. BA p value: 		 0.721037342023
2 hours east traveled away bat. OBP p value: 		 0.678689369224
2 hours east traveled away bat. SLG p value: 		 0.46656601949
2 hours east traveled away bat. ISO p value: 		 0.403195477521
2 hours east traveled away pit. H/9 p value: 		 0.416652227442
2 hours east traveled away pit. BB/9 p value: 		 0.47226387587
2 hours east traveled away pit. K/9 p value: 		 0.330684091695
2 hours east traveled away pit. ERA p value: 		 0.0988118924438
2 hours east traveled away pit. FIP p value: 		 0.827145520646


In [26]:
print ("p values at 3 time zones traveled east")
print ("--------------------------------------")
print ("3 hours east traveled count: \t\t\t\t",len(dfGameData3hourseast.index))
print ("3 hours east traveled away bat. R/9 p value: \t\t",(stats.ttest_ind(dfGameData3hourseast['away Batting R/9'], dfGameData['away Batting R/9'])[1]))
print ("3 hours east traveled away bat. H/9 p value: \t\t",(stats.ttest_ind(dfGameData3hourseast['away Batting H/9'], dfGameData['away Batting H/9'])[1]))
print ("3 hours east traveled away bat. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData3hourseast['away Batting BB/9'], dfGameData['away Batting BB/9'])[1]))
print ("3 hours east traveled away bat.  K/9 p value: \t\t",(stats.ttest_ind(dfGameData3hourseast['away Batting K/9'], dfGameData['away Batting K/9'])[1]))
print ("3 hours east traveled away bat. BA p value: \t\t",(stats.ttest_ind(dfGameData3hourseast['awayBA'], dfGameData['awayBA'])[1]))
print ("3 hours east traveled away bat. OBP p value: \t\t",(stats.ttest_ind(dfGameData3hourseast['awayOBP'], dfGameData['awayOBP'])[1]))
print ("3 hours east traveled away bat. SLG p value: \t\t",(stats.ttest_ind(dfGameData3hourseast['awaySLG'], dfGameData['awaySLG'])[1]))
print ("3 hours east traveled away bat. ISO p value: \t\t",(stats.ttest_ind(dfGameData3hourseast['awayISO'], dfGameData['awayISO'])[1]))
print ("3 hours east traveled away pit. H/9 p value: \t\t",(stats.ttest_ind(dfGameData3hourseast['away Pitching H/9'], dfGameData['away Pitching H/9'])[1]))
print ("3 hours east traveled away pit. BB/9 p value: \t\t",(stats.ttest_ind(dfGameData3hourseast['away Pitching BB/9'], dfGameData['away Pitching BB/9'])[1]))
print ("3 hours east traveled away pit. K/9 p value: \t\t",(stats.ttest_ind(dfGameData3hourseast['away Pitching K/9'], dfGameData['away Pitching K/9'])[1]))
print ("3 hours east traveled away pit. ERA p value: \t\t",(stats.ttest_ind(dfGameData3hourseast['awayERA'], dfGameData['awayERA'])[1]))
print ("3 hours east traveled away pit. FIP p value: \t\t",(stats.ttest_ind(dfGameData3hourseast['awayFIP'], dfGameData['awayFIP'])[1]))

p values at 3 time zones traveled east
--------------------------------------
3 hours east traveled count: 				 5257
3 hours east traveled away bat. R/9 p value: 		 0.0792366724925
3 hours east traveled away bat. H/9 p value: 		 0.265168512639
3 hours east traveled away bat. BB/9 p value: 		 0.000128133733127
3 hours east traveled away bat.  K/9 p value: 		 0.00598510546619
3 hours east traveled away bat. BA p value: 		 0.228519459963
3 hours east traveled away bat. OBP p value: 		 0.00635551807289
3 hours east traveled away bat. SLG p value: 		 0.437633154234
3 hours east traveled away bat. ISO p value: 		 0.816360392344
3 hours east traveled away pit. H/9 p value: 		 0.0915598589645
3 hours east traveled away pit. BB/9 p value: 		 0.000193724573059
3 hours east traveled away pit. K/9 p value: 		 1.50046846153e-05
3 hours east traveled away pit. ERA p value: 		 0.118625322703
3 hours east traveled away pit. FIP p value: 		 0.242795019794
